<a href="https://colab.research.google.com/github/Nurdaylight/An-Econ-771/blob/main/PS2/ps2_Nurdaulet_Menglibayev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
# Import necessary packages
import pandas as pd
import numpy as np
from scipy.linalg import block_diag


In [13]:
ps2_data = pd.read_stata("http://fmwww.bc.edu/ec-p/data/Hayashi/nerlove63.dta")
ps2_data=np.log(ps2_data)
ps2_data['const'] = 1
ps2_data = ps2_data[['totcost', 'const', 'output', 'plabor', 'pkap', 'pfuel']]
ps2_data.head(3)

,totcost,const,output,plabor,pkap,pfuel
0,-2.501036,1,0.693147,0.737164,5.209486,2.884801
1,-0.414001,1,1.098612,0.717840,5.159055,3.558201
2,-0.010050,1,1.386294,0.717840,5.141664,3.558201


In [118]:
#Print function for nicer look
def print_estimates(betas, ses):
    for i, (b, se) in enumerate(zip(betas.flatten(), ses.flatten()), 1):
        print(f"β{i-1}: {b:>10.3f}   SE: {se:>10.3f}")
letters = "ABCDEFGHI"
def print_estimates1(betas, ses):
    b = betas.flatten()
    s = ses.flatten()
    n = len(b)

    z = n // 4
    b = b.reshape(z, 4)

    s = s.reshape(z, 4)
    for j in range(5):
      for i in range(4):

            print(f"β{letters[j]},{i}: {b[j,i]:>10.3f}   SE: {s[j,i]:>10.3f}")



In [79]:
b4.reshape(4, 5)

array([[-3.34334588,  0.40029039,  0.61517388, -0.08135592, -6.48897341],
       [ 0.65815086,  0.09379981,  0.37793558, -7.3329412 ,  0.93827933],
       [ 0.40225789,  0.25000834, -6.54604882,  0.91204404,  0.50695742],
       [ 0.0933525 , -6.71425709,  1.0443901 ,  0.60258858, -0.28943563]])

### a

### b

In [23]:
#Preparing data
X = ps2_data.iloc[:, 1:].to_numpy()          # shape: (n, k)
Y = ps2_data.iloc[:, 0].to_numpy().reshape(-1, 1)

# Point estimates
inv1 =np.linalg.inv(X.T@X)
b1=inv1@ X.T@Y
# Standard errors
e=Y-X@b1
var1 = ((e.T @ e) / (140))[0] * inv1
se1= np.sqrt(np.diag(var1)).reshape(-1, 1)
print_estimates(b1, se1)



β0:     -3.527   SE:      1.774
β1:      0.720   SE:      0.017
β2:      0.436   SE:      0.291
β3:     -0.220   SE:      0.339
β4:      0.427   SE:      0.100


### c

In [27]:
X2 = X[:, 0:4].copy()
X2[:, 2:4] = X2[:, 2:4] - (X[:, 4])[:, None]

Y2=Y-(X[:,4])[:, None]


In [29]:
#restricted regression
inv2= np.linalg.inv(X2.T@X2)
b2=inv2@X2.T@Y2
out2=np.round(b2,3)

#restricted STD
e2=Y2-X2@b2
var2 = ((e2.T @ e2) / (141))[0] * inv2
se2= np.sqrt(np.diag(var2)).reshape(-1, 1)
print_estimates(b2, se2)


β0:     -4.691   SE:      0.885
β1:      0.721   SE:      0.017
β2:      0.593   SE:      0.205
β3:     -0.007   SE:      0.191


### **Answer:**


*   Yes, I am able to replicate Nerlove's results. However replication is not completely exact. There is slight margin of error for reasons specified in the question.
*   These results in his paper appear on **Table 3** Row I






### d

In [123]:
#datasplit
X3=[]
Y3=[]
for i in range(5):
  print("                                                          ")
  print("----------------------------------------------------->")
  print(f"Group results of {letters[i]}:")
  X2_1=X2[29*i:29*(i+1)]
  Y2_1=Y2[29*i:29*(i+1)]
  X3.append(X2_1)
  Y3.append(Y2_1)
  #restricted regression
  inv3=np.linalg.inv(X2_1.T@X2_1)
  estim2=inv3@X2_1.T@Y2_1
  #restricted STD
  e2=Y2_1-X2_1@estim2
  var3 = ((e2.T @ e2) / (25))[0] * inv3
  se_estim2= np.sqrt(np.diag(var3 )).reshape(-1, 1)
  print_estimates(estim2, se_estim2)

                                                          
----------------------------------------------------->
Group results of A:
β0:     -3.343   SE:      3.146
β1:      0.400   SE:      0.084
β2:      0.615   SE:      0.729
β3:     -0.081   SE:      0.706
                                                          
----------------------------------------------------->
Group results of B:
β0:     -6.489   SE:      1.413
β1:      0.658   SE:      0.116
β2:      0.094   SE:      0.274
β3:      0.378   SE:      0.277
                                                          
----------------------------------------------------->
Group results of C:
β0:     -7.333   SE:      1.689
β1:      0.938   SE:      0.198
β2:      0.402   SE:      0.199
β3:      0.250   SE:      0.187
                                                          
----------------------------------------------------->
Group results of D:
β0:     -6.546   SE:      1.165
β1:      0.912   SE:      0.107
β2:      0.507  

In [119]:
# Use Y2 regress it on diagonal of X

X4 = block_diag(*X3)

# Point estimates
inv4 =np.linalg.inv(X4.T@X4)
b4=inv4@ X4.T@Y2
# Standard errors
e=Y2-X4@b4
var4 = ((e.T @ e) / (140))[0] * inv4
se4= np.sqrt(np.diag(var4)).reshape(-1, 1)
print_estimates1(b4, se4)


βA,0:     -3.343   SE:      1.562
βA,1:      0.400   SE:      0.042
βA,2:      0.615   SE:      0.362
βA,3:     -0.081   SE:      0.351
βB,0:     -6.489   SE:      1.725
βB,1:      0.658   SE:      0.142
βB,2:      0.094   SE:      0.335
βB,3:      0.378   SE:      0.338
βC,0:     -7.333   SE:      2.525
βC,1:      0.938   SE:      0.296
βC,2:      0.402   SE:      0.298
βC,3:      0.250   SE:      0.280
βD,0:     -6.546   SE:      2.859
βD,1:      0.912   SE:      0.264
βD,2:      0.507   SE:      0.460
βD,3:      0.093   SE:      0.403
βE,0:     -6.714   SE:      2.061
βE,1:      1.044   SE:      0.128
βE,2:      0.603   SE:      0.389
βE,3:     -0.289   SE:      0.344


array([[-3.34334588],
       [ 0.40029039],
       [ 0.61517388],
       [-0.08135592],
       [-6.48897341],
       [ 0.65815086],
       [ 0.09379981],
       [ 0.37793558],
       [-7.3329412 ],
       [ 0.93827933],
       [ 0.40225789],
       [ 0.25000834],
       [-6.54604882],
       [ 0.91204404],
       [ 0.50695742],
       [ 0.0933525 ],
       [-6.71425709],
       [ 1.0443901 ],
       [ 0.60258858],
       [-0.28943563]])

In [110]:

b = b4.flatten()
s = se4.flatten()
n = len(b)
z = n // 4
b = b.reshape(z, 4)
b

array([[-3.34334588,  0.40029039,  0.61517388, -0.08135592],
       [-6.48897341,  0.65815086,  0.09379981,  0.37793558],
       [-7.3329412 ,  0.93827933,  0.40225789,  0.25000834],
       [-6.54604882,  0.91204404,  0.50695742,  0.0933525 ],
       [-6.71425709,  1.0443901 ,  0.60258858, -0.28943563]])

In [105]:
 z = n // 4
    b = b.reshape(4, z)

    s = s.reshape(4, z)
    for j in range(5):
      for i in range(4):

            print(f"β{j},{i}: {b[j,i]:>10.3f}   SE: {s[j,i]:>10.3f}")

IndentationError: unexpected indent (ipython-input-3013974656.py, line 2)